# Calculate Precipitation-Buoyancy POD

This notebook calculates the precipitation-buoyancy relationship POD as defined in [Ahmed and Neelin (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021GL094108).

## Import Necessary Packages

In [1]:
import warnings
import numpy as np
import xarray as xr
from datetime import datetime
warnings.filterwarnings('ignore')

## User-Defined Configurations

Define the user's name/email, specify the directory where the observational datasets are, and set the directory where the P-$B_L$ datasets will be saved.

In [2]:
AUTHOR  = 'Savannah L. Ferretti'
EMAIL   = 'savannah.ferretti@uci.edu'
FILEDIR = '/global/cfs/cdirs/m4334/sferrett/monsoon-pod/data/interim'
SAVEDIR = '/global/cfs/cdirs/m4334/sferrett/monsoon-pod/data/processed'

## Load Baseline Datasets

Load in the baseline datasets (from `FILEDIR`) which contain precipitation (mm/day), surface pressure ($p_s$, hPa), temperature ($T$, K), and specific humidity ($q$, kg/kg) data. Before loading, create a 4D pressure variable ($p$, hPa) using the `add_p_array()` function, based on the vertical levels of $T$ and $q$, and incorporate it into the dataset.

In [3]:
def add_p_array(ds):
    p = ds.lev.expand_dims({'time':ds.time,'lat':ds.lat,'lon':ds.lon}).transpose('lev','time','lat','lon')
    p.name = 'p'
    ds = ds.assign(p=p)
    return ds

def load(filename,filedir=FILEDIR):
    filepath = f'{filedir}/{filename}'
    ds = xr.open_dataset(filepath)
    ds = add_p_array(ds)
    return ds.load()

In [4]:
hrimerg = load('HR_ERA5_IMERG_baseline.nc')
lrimerg = load('LR_ERA5_IMERG_baseline.nc')
lrgpcp  = load('LR_ERA5_GPCP_baseline.nc')

## Functions to Calculate $B_L$

### Calculate Equivalent Potential Temperature

The `calc_thetae()` function calculates equivalent potential temperature ($\theta_e$, K) following [Bolton (1980)](https://journals.ametsoc.org/view/journals/mwre/108/7/1520-0493_1980_108_1046_tcoept_2_0_co_2.xml?tab_body=pdf) Equation 43 (where $T_L$ is given by Equation 55):
    
$$ \theta_e = T\left(\frac{1000}{p}\right)^{0.2854\left(1-0.28 \times 10^{-3}r\right)} \times \text{ exp}\left[\left(\frac{3.376}{T_L}-0.00254\right) \times r\left(1+0.81 \times 10^{-3}r\right)\right]$$

Saturated equivalent potential temperature ($\theta^*_e$) is calculated by substituting $q$ for its saturated counterpart ($q_s$). The `calc_qs()` function calculates $q_s$ following Equation 4 from [Plymouth State Weather Center (2018)](https://vortex.plymouth.edu/~stmiller/stmiller_content/Publications/AtmosRH_Equations_Rev.pdf), where saturation vapor pressure ($e_s$, hPa) is calculated using the `calc_es()` function, following Equations 17 and 18 from [Kuang (2018)](https://journals.ametsoc.org/view/journals/apme/57/6/jamc-d-17-0334.1.xml).

In [6]:
def calc_es(t):
    tc = t-273.15
    eswat = np.exp(34.494-(4924.99/(tc+237.1)))/((tc+105.)**1.57)
    esice = np.exp(43.494-(6545.8/(tc+278.)))/((tc+868.)**2.)
    es = np.where(tc>0.,eswat,esice) 
    es = es/100. # Pa to hPa
    return es

def calc_qs(p,t):
    rv = 461.50   
    rd = 287.04    
    es = calc_es(t) 
    epsilon = rd/rv
    qs = (epsilon*es)/(p-es*(1.-epsilon))
    return qs

def calc_thetae(p,t,q):
    p0 = 1000.  
    rv = 461.5  
    rd = 287.04
    epsilon = rd/rv
    r  = q/(1.-q) 
    e  = (t.lev*r)/(epsilon+r)
    tl = 2840./(3.5*np.log(t)-np.log(e)-4.805)+55.
    thetae = t*(p0/p)**(0.2854*(1.-0.28*r))*np.exp((3.376/tl-0.00254)*1000.*r*(1.+0.81*r))
    return thetae

### Calculate Layer Averages

The boundary layer is defined as a 100 hPa thick layer above the surface, and the lower free-troposphere is defined as the layer extending from the top of the boundary layer to 500 hPa. Calculate the average $\theta_e$ over the boundary layer ($\theta_{eB}$) and lower free-troposphere ($\theta_{eL}$),  as well as ${\theta^*_e}$ over the lower-free troposphere (${\theta^*_{eL}}$). The `get_p_above()` and `get_p_below()` functions find the upper and lower bounds of the specified atmospheric layer, respectively, and `calc_layer_average()` uses numerical integration to calculate the average within that layer.

In [7]:
def get_p_above(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    pabove = levels[np.maximum(idx-1,0)]
    return pabove

def get_p_below(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    pbelow = levels[np.minimum(idx,len(levels)-1)]
    return pbelow

def calc_layer_average(data,a,b):
    a,b,data = a.load(),b.load(),data.load()
    pabove = xr.apply_ufunc(get_p_above,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    pbelow = xr.apply_ufunc(get_p_below,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fabove/2*(pbelow-pabove)*(a<data.lev[-1])
    pbelow += (pbelow==pabove) 
    belowintegral = (a-pabove)*fabove+(fbelow-fabove)*(a-pabove)**2/(pbelow-pabove)/2+correction
    innerintegral = (data*(data.lev<=a)*(data.lev>=b)).integrate('lev')
    pabove = xr.apply_ufunc(get_p_above,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    pbelow = xr.apply_ufunc(get_p_below,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fbelow/2*(pbelow-pabove)*(b>data.lev[0])
    pabove -= (pbelow==pabove) 
    aboveintegral = (pbelow-b)*fabove+(fbelow-fabove)*(pbelow-pabove)*(1-((b-pabove)/(pbelow-pabove))**2)/2+correction
    layeraverage  = (belowintegral+innerintegral+aboveintegral)/(a-b)
    return layeraverage

### Calculate $B_L$

The `calc_weights()` function calculates the weighted contributions of the boundary layer ($w_B$) and lower free-troposphere ($w_L$) following [Adames et al. (2021)](https://journals.ametsoc.org/view/journals/atsc/78/2/jas-d-20-0074.1.xml) Equations 5a and 5b:

$$ w_B = \frac{\Delta p_B}{\Delta p_L} \ln{\left(1 + \frac{\Delta p_L}{\Delta p_B}\right)} $$

$$ w_L = 1 - w_B $$

In [8]:
def calc_weights(ps,pbltop,lfttop):
    pblthickness = ps-pbltop
    lftthickness = pbltop-lfttop
    wb = (pblthickness/lftthickness)*np.log((pblthickness+lftthickness)/pblthickness)
    wl = 1-wb
    return wb,wl

The `calc_bl_terms()` function follows [Ahmed and Neelin (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021GL094108) Equation 1, where the average buoyancy in the lower troposphere ($B_L$) is calculated as:

$$ B_L = \frac{g}{\overline{\kappa}_L\theta_{e0}}\left[w_B\underbrace{\left(\frac{\theta_{eB}-\theta^{*}_{eL}}{\theta^{*}_{eL}}\right)\theta_{e0}}_{\mathrm{CAPE_L}} - w_L\underbrace{\left(\frac{\theta^{*}_{eL}-\theta_{eL}}{\theta^{*}_{eL}}\right)\theta_{e0}}_{\mathrm{SUBSAT_L}}\right] $$

In [9]:
def calc_bl_terms(thetaeb,thetael,thetaels,wb,wl):
    g       = 9.81
    kappal  = 3.
    thetae0 = 340.
    cape    = ((thetaeb-thetaels)/thetaels)*thetae0
    subsat  = ((thetaels-thetael)/thetaels)*thetae0
    bl      = (g/(kappal*thetae0))*((wb*cape)-(wl*subsat))
    return cape,subsat,bl

We use the `create_dataset()` function to put all calculated variables into a singular Xarray.Dataset.

In [10]:
def create_dataset(wb,wl,cape,subsat,bl,pr,author=AUTHOR,email=EMAIL):
    vardata = {'wb':([*wb.dims],wb.data),
               'wl':([*wl.dims],wl.data),
               'cape':([*cape.dims],cape.data),
               'subsat':([*subsat.dims],subsat.data),
               'bl':([*bl.dims],bl.data),
               'pr':([*pr.dims],pr.data)}
    coords  = {'time':pr.time.data,'lat':pr.lat.data,'lon':pr.lon.data}
    ds = xr.Dataset(vardata,coords)
    ds.wb.attrs     = dict(long_name='Fractional contribution of the boundary layer',units='0-1')
    ds.wl.attrs     = dict(long_name='Fractional contribution of the lower free-troposphere',units='0-1')
    ds.cape.attrs   = dict(long_name='Undilute plume buoyancy',units='K')
    ds.subsat.attrs = dict(long_name='Subsaturation in the lower-free troposphere',units='K')
    ds.bl.attrs     = dict(long_name='Average buoyancy in the lower troposphere',units='m/s²')
    ds.pr.attrs     = dict(long_name='Precipitation rate',units='mm/day')
    ds.time.attrs   = dict(long_name='Time')
    ds.lat.attrs    = dict(long_name='Latitude',units='$^\circ$N')
    ds.lon.attrs    = dict(long_name='Longitude',units='$^\circ$E')
    ds.attrs        = dict(description='Calculated following Ahmed & Neelin (2021) Eq. 1',
                           history=f'Created on {datetime.today().strftime("%Y-%m-%d")} by {author} ({email})')
    return ds

## Execute $B_L$ Calculation

Since the analysis is quite compute-intensive (particularly for the high-resolution data), we execute the aforementioned workflow by year. The `process_by_year()` function creates the yearly P-$B_L$ datasets, and merges them into a single Xarray.Dataset.

In [12]:
def process_by_year(ds,author=AUTHOR,email=EMAIL):
    dslist = []
    for year in np.unique(ds.time.dt.year.values):
        data     = ds.sel(time=ds.time.dt.year==year)
        qs       = calc_qs(data.p,data.t)
        thetae   = calc_thetae(data.p,data.t,data.q)
        thetaes  = calc_thetae(data.p,data.t,qs)
        pbltop   = data.ps-100.
        lfttop   = xr.full_like(data.ps,500.) 
        thetaeb  = calc_layer_average(thetae,data.ps,pbltop)*np.sqrt(-1+2*(data.ps>lfttop))
        thetael  = calc_layer_average(thetae,pbltop,lfttop)
        thetaels = calc_layer_average(thetaes,pbltop,lfttop)
        wb,wl    = calc_weights(data.ps,pbltop,lfttop)
        blterms  = calc_bl_terms(thetaeb,thetael,thetaels,wb,wl)
        newds    = create_dataset(wb,wl,blterms[0],blterms[1],blterms[2],data.pr,author,email)  
        dslist.append(newds)
    return xr.concat(dslist,dim='time')

In [18]:
hrimergprbl = process_by_year(hrimerg)
lrimergprbl = process_by_year(lrimerg)
lrgpcpprbl  = process_by_year(lrgpcp)

## Save P-$B_L$ Datasets

Save each P-$B_L$ Xarray.Dataset as a netCDF file to `SAVEDIR`.

In [20]:
def save(ds,filename,savedir=SAVEDIR):
    filepath = f'{savedir}/{filename}'
    ds.to_netcdf(filepath)

In [22]:
save(hrimergprbl,'HR_ERA5_IMERG_pr_bl_terms.nc')
save(lrimergprbl,'LR_ERA5_IMERG_pr_bl_terms.nc')
save(lrgpcpprbl,'LR_ERA5_GPCP_pr_bl_terms.nc')